# ENSEMBLE MODEL TABLE OF CONTENT
- **Uploader**
- **Data Prep**
- **Categorical Data Prep and Modeling**
  - ADHD Model - Oversampled GBDT
  - Sex_F Model - Hybrid GBDT
- **Quantitative Data Prep and Modeling**
- **FCM Data Prep and Modeling**
- **Emsemble Modeling - Stacking Method**
- **Kaggle Submission Maker**

*Maya: I am up to getting best models for Cat rn, if anyone wants to find and input the best models along with their dataprep (dataset modifications) that would be great!*

In [1]:
#@title uploader
file_id = "1VpSCc_ms2dj59bMAAOqCu0OI0jjb-8Hq" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

widsdatathon2025.zip
Archive:  widsdatathon2025.zip
  inflating: Data Dictionary.xlsx    
  inflating: SAMPLE_SUBMISSION.xlsx  
  inflating: TEST/TEST_CATEGORICAL.xlsx  
  inflating: TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv  
  inflating: TEST/TEST_QUANTITATIVE_METADATA.xlsx  
  inflating: TRAIN/TRAINING_SOLUTIONS.xlsx  
  inflating: TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx  
  inflating: TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv  
  inflating: TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx  


## Data Prep

In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install scikit-learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [9]:
#all datasets
train_quantitative = pd.read_excel('TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx')
train_answers = pd.read_excel('TRAIN/TRAINING_SOLUTIONS.xlsx')
train_categorical = pd.read_excel("TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
train_solutions = pd.read_excel("TRAIN/TRAINING_SOLUTIONS.xlsx")
fcm = pd.read_csv("TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv")
fcm_solutions = pd.read_excel("TRAIN/TRAINING_SOLUTIONS.xlsx")
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")

## Categorical Data Prep + Modeling
### ADHD - Oversampling gradient boosted trees (gave worse accuracy?)


In [58]:
#drop id
for feature in train_categorical.drop(columns = ['participant_id']).columns:
  print(train_categorical[feature].value_counts())
  print("\n")

Basic_Demos_Enroll_Year
2018    415
2019    312
2017    254
2016    191
2015     36
2020      5
Name: count, dtype: int64


Basic_Demos_Study_Site
1    652
3    430
4    120
2     11
Name: count, dtype: int64


PreInt_Demos_Fam_Child_Ethnicity
0.0    809
1.0    296
2.0     77
3.0     20
Name: count, dtype: int64


PreInt_Demos_Fam_Child_Race
0     627
8     195
1     181
2     128
3      30
9      23
10     11
4      10
11      6
7       2
Name: count, dtype: int64


MRI_Track_Scan_Location
2    532
3    463
1    179
4     36
0      3
Name: count, dtype: int64


Barratt_Barratt_P1_Edu
21    470
18    421
15    162
12     97
9      28
0      15
6      15
3       5
Name: count, dtype: int64


Barratt_Barratt_P1_Occ
0     317
35    219
45    187
40    154
30    154
25     79
15     35
20     31
5      31
10      6
Name: count, dtype: int64


Barratt_Barratt_P2_Edu
21    323
18    301
0     198
15    166
12    162
9      44
6      14
3       5
Name: count, dtype: int64


Barratt_Barratt_P2

In [60]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')

participant_ids = df['participant_id']

In [61]:
#fill missing NA values
df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':df['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)

In [62]:
X = df.drop(columns=['Sex_F', 'participant_id'])
y = df['Sex_F']

In [63]:
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [64]:
cat_balanced = pd.DataFrame(X_resampled, columns=X.columns)
cat_balanced['Sex_F'] = y_resampled

In [65]:
cat_balanced.to_csv('cat_balanced_over.csv', index=False)

Running grid search got:

{'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
Validation accuracy: 0.8713725490196078

In [66]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# Example: assuming X and y are already defined
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize GBDT model with best parameters
gbdt_best = GradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=3,
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=500,
    random_state=42
)

# Train the model
gbdt_best.fit(X_train, y_train)

# Evaluate on validation set
y_pred = gbdt_best.predict(X_val)

# Accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Weighted F1 Score
f1 = f1_score(y_val, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")


Validation Accuracy: 0.6831
Weighted F1 Score: 0.6782


### Sex_f - hybrid gradient boosting trees

In [50]:
from imblearn.combine import SMOTETomek

In [51]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')

participant_ids = df['participant_id']

In [52]:
#fill missing NA values
df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':df['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)

In [53]:
X = df.drop(columns=['Sex_F', 'participant_id'])
y = df['Sex_F']

In [54]:
smt = SMOTETomek(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smt.fit_resample(X, y)

In [55]:
cat_balanced = pd.DataFrame(X_resampled, columns=X.columns)
cat_balanced['Sex_F'] = y_resampled

In [56]:
cat_balanced.to_csv('cat_balanced_hybrid.csv', index=False)

Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Validation accuracy: 0.9338983050847458

In [57]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Define features and labels
X = cat_balanced.drop(['ADHD_Outcome', 'Sex_F'], axis=1)
y = cat_balanced['Sex_F']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize Gradient Boosting Classifier with best parameters
gbdt_final = GradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=5,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=200,
    random_state=42
)

# Train the model
gbdt_final.fit(X_train, y_train)

# Predict on test set
y_pred = gbdt_final.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Weighted F1 Score: {f1:.4f}")


Test Accuracy: 0.7162
Weighted F1 Score: 0.7162


##Quantitative Data Prep


### ADHD - Gradient Boosted DT

{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

### Sex_f - Optuna Model

'n_estimators': 150, 'learning_rate': 0.08374413872051761, 'max_depth': 4


## FCM Data Prep